## Create Model Directory tructure

In [1]:

!mkdir -p models/huggingface-models
!mkdir -p models/huggingface-model/1

## Define a HuggingFace Pre-Trained Model

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 47.8 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.1 MB/s 


In [ ]:
import torch
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer


R_tokenizer = XLMRobertaTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')
premise = "Jupiter's Biggest Moons Started as Tiny Grains of Hail"
hypothesis = 'This text is about space & cosmos'

input_ids = R_tokenizer.encode(premise, hypothesis, return_tensors='pt', 
                               max_length=256, padding='max_length')

mask = input_ids != 1
mask = mask.long()


class PyTorch_to_TorchScript(torch.nn.Module):
    def __init__(self):
        super(PyTorch_to_TorchScript, self).__init__()
        self.model = XLMRobertaForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli', return_dict=False)
    def forward(self, data, attention_mask=None):
        return self.model(data.cuda(), attention_mask.cuda())

pt_model = PyTorch_to_TorchScript().eval().cuda()
traced_script_module = torch.jit.trace(pt_model, (input_ids, mask))
traced_script_module.save('models/huggingface-model/1/model.pt')
